In [55]:
import h5py
from sklearn.model_selection import train_test_split
import numpy as np

In [56]:
data = h5py.File('data/Q1/MNIST_Subset.h5','r+')
X=data['X'][:]
Y=data['Y'][:]

In [57]:
X.shape,Y.shape

((14251, 28, 28), (14251,))

In [58]:
X = X.reshape(X.shape[0],-1)

In [59]:
X=X/255

In [60]:
np.unique(Y)

array([7, 9], dtype=uint8)

In [61]:
#as we only have 2 classes
Y = np.where(Y==7,0,1)

In [63]:
n_classes = len(np.unique(Y))
targets = Y.reshape(-1)
Y = np.eye(n_classes)[targets]

In [65]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.2, random_state=42)

In [66]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

((11400, 784), (2851, 784), (11400, 2), (2851, 2))

In [70]:
#hyperparameters
epoch = 10
learning_rate = 0.01

In [71]:
def sigmoid(z,derative = False):
    if derative:
        return z*(1-z)
    return 1.0/(1.0+np.exp(-z))

In [85]:
def init_weights(n_hidden_layers, number_of_neurons):
    weights_layers =[np.random.uniform(size=(X_train.shape[1],number_of_neurons[0]))]
    bais_layers = [np.random.uniform(size=(1,number_of_neurons[0]))]
    
    for i in range(1,n_hidden_layers):
        weights_layers.append(np.random.uniform(size=(weights_layers[i-1].shape[1],number_of_neurons[i])))
        bais_layers.append(np.random.uniform(size=(1,number_of_neurons[i])))
    
    weights_layers.append(np.random.uniform(size=(weights_layers[-1].shape[1],number_of_class)))
    bais_layers.append(np.random.uniform(size=(1,number_of_class)))
    
    return weights_layers,bais_layers

In [86]:
weight_layers,bais_layers = init_weights(2,[100,50])

In [87]:
len(weight_layers)

3

In [89]:
for i,j in zip(weight_layers,bais_layers):
    print(i.shape,j.shape)

(784, 100) (1, 100)
(100, 50) (1, 50)
(50, 2) (1, 2)
